In [1]:
import pymysql
import pandas as pd
import numpy as np

In [2]:
conn = None
cur = None

sql = ""

In [3]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='ehgus1319@', db='mydb', charset='utf8' )
cur = conn.cursor()

In [4]:
def search_user_data(conn):
    sql = 'SELECT * FROM user'
    cur.execute(sql)
    results = cur.fetchall()
    df = pd.DataFrame(results, columns=['user_id', 'username', 'password', 'enabled', 'MBTI', 'SEX', 'AGE'])
#     users = df.set_index('id')
    return df;

In [5]:
def search_webtoon_data(conn):
    sql = 'SELECT * FROM webtoon_data'
    cur.execute(sql)
    results = cur.fetchall()
    df = pd.DataFrame(results, columns=['title_id', 'title_name', 'author', 'day', 'genre', 'story', 'link', 'img_src'])
    webtoons = df.set_index('title_id')
    return webtoons;

In [6]:
def search_rating_data(conn):
    sql = 'SELECT * FROM rating'
    cur.execute(sql)
    results = cur.fetchall()
    df = pd.DataFrame(results, columns=['id', 'user_id', 'webtoon_id', 'rating'])
    return df;

In [11]:
def search_recommendation(conn):
    sql = 'SELECT * FROM recommendation'
    cur.execute(sql)
    results = cur.fetchall()
    df = pd.DataFrame(results, columns=['id','user', 'webtoonid', 'rank'])
    return df;

In [12]:
search_user_data(conn).head()

,user_id,username,password,enabled,MBTI,SEX,AGE
0,5,1,$2a$10$tw7VkAGfz28t6wJyPdRfgelqFfqYNGLhbKtONIg...,b'\x01',ISTJ,Male,26
1,7,2,$2a$10$BJ.BaHjto.UEUNG7BcC3juHm/shVIMuxQCuJ8D1...,b'\x01',INFP,Male,26
2,9,3,$2a$10$VKuwa1eFx1E48.x2kFWgSue1bNFjPPC9WmGH/T3...,b'\x01',ENFP,Female,23
3,11,4,$2a$10$XTlPnAmR54V.Ji7DjEaj9O4VS9z05XOI1xnkkzz...,b'\x01',ESFJ,Male,22
4,12,5,$2a$10$xkexm4JzxedgEczLIOj6eefytH4Y76ARHz2laXm...,b'\x01',INTJ,Male,23


In [13]:
search_recommendation(conn).head()

,id,user,webtoonid,rank
0,1,5,784248,1
1,2,5,790713,2
2,3,5,776601,3
3,4,5,730694,4
4,5,5,739115,5


In [14]:
search_webtoon_data(conn).head()

,title_name,author,day,genre,story,link,img_src
title_id,,,,,,,
21815,히어로메이커,빤쓰,월,"옴니버스, 판타지",왕은 영웅이 되고 싶어하는 공주의 소원을 들어주기로 전격 결심! 공주를 속이고 마치...,https://comic.naver.com/webtoon/list?titleId=2...,https://shared-comic.pstatic.net/thumb/webtoon...
64997,나이트런,김성민,토,"스토리, 판타지",우주력 430년. 성간이동이 가능해져 별과 별을 이동하는 시대\n인간은 괴수와 싸우...,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...
131385,쿠베라,카레곰,목,"스토리, 판타지",신의 이름을 가진 소녀 쿠베라 리즈. 15세 생일에 외출에서 돌아오다가 초토화되어버...,https://comic.naver.com/webtoon/list?titleId=1...,https://shared-comic.pstatic.net/thumb/webtoon...
318995,갓 오브 하이스쿨,박용제,금,"스토리, 판타지","전국, 전세계 고등학생 중 가장 쎈 녀석을 뽑는 대회가 열린다.\n허구 100% 막...",https://comic.naver.com/webtoon/list?titleId=3...,https://shared-comic.pstatic.net/thumb/webtoon...
400739,에이머,구동인,화,"스토리, 판타지","작은 우연은 역사를 만든다.\n지구를 침공한 이성인 팜킨 일당과, 그들을 막아선 초...",https://comic.naver.com/webtoon/list?titleId=4...,https://shared-comic.pstatic.net/thumb/webtoon...


In [15]:
search_rating_data(conn).head()

,id,user_id,webtoon_id,rating
0,16,5,64997,3
1,17,5,318995,3
2,18,5,570503,5
3,19,5,552960,2
4,20,5,597447,4


In [16]:
user = search_user_data(conn)
webtoons = search_webtoon_data(conn)
ratings = search_rating_data(conn)
recommendation = search_recommendation(conn)

#단순 평점 높은순
def recom_webtoon(n_items):
    webtoon_mean = ratings.groupby(['webtoon_id'])['rating'].mean()
    webtoon_sort = webtoon_mean.sort_values(ascending=False)[:n_items]
    recom_webtoons = webtoons.loc[webtoon_sort.index]
    recommendations = recom_webtoons['title_name']
    return recommendations

recom_webtoon(20)

webtoon_id
773524        거래하실래요?
795262           사형소년
717481           일렉시드
724815        아홉수 우리들
784248      내가 키운 S급들
783769     내 남편과 결혼해줘
737628           별이삼샵
739115           앵무살수
780191        선남친 후연애
778963             덴큐
778580           최강부캐
777767     역대급 영지 설계사
776601           광마회귀
746858    이번 생도 잘 부탁해
747271           나노마신
748105           독립일기
751168            상남자
769568       반드시 해피엔딩
765804        A.I. 닥터
790713       대학원 탈출일지
Name: title_name, dtype: object

In [17]:
# users = user.set_index('user_id')
users = user[['user_id', 'MBTI', 'AGE', 'SEX']]
ratings = ratings[['user_id','webtoon_id', 'rating']]

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

conn = pymysql.connect(host='127.0.0.1', user='root', password='ehgus1319@', db='mydb', charset='utf8' )
cur = conn.cursor()
ratings = search_rating_data(conn)
ratings = ratings[['user_id','webtoon_id', 'rating']]

x = ratings.copy()
y = ratings['user_id']

user = users.copy()

for i in range(len(user)):
    if 'TJ' in user.loc[i, 'MBTI']:
        user.loc[i, 'MBTI'] = 0.1
    elif 'FJ' in user.loc[i, 'MBTI']:
        user.loc[i, 'MBTI'] = 0.2
    elif 'FP' in user.loc[i, 'MBTI']:
        user.loc[i, 'MBTI'] = 0.3
    elif 'TP' in user.loc[i, 'MBTI']:
        user.loc[i, 'MBTI'] = 0.4    
    
#     if user.loc[i, 'SEX'] == 'Male':
#         user.loc[i, 'SEX'] = 0.5
#     else:
#         user.loc[i, 'SEX'] = 1
        
    if user.loc[i, 'AGE'] / 10 < 2:
        user.loc[i, 'AGE'] = 0.1
    elif user.loc[i, 'AGE'] / 20 < 2:
        user.loc[i, 'AGE'] = 0.2
    elif user.loc[i, 'AGE'] / 30 < 2:
        user.loc[i, 'AGE'] = 0.3
    elif user.loc[i, 'AGE'] / 40 < 2:
        user.loc[i, 'AGE'] = 0.4
    elif user.loc[i, 'AGE'] / 50 < 2:
        user.loc[i, 'AGE'] = 0.5
        
user_matrix = user.pivot(index = 'user_id',
                        columns = 'SEX',
                        values = 'MBTI')


  
x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                    test_size = 0.25)
                                                    
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix = x_train.pivot(index = 'user_id',
                              columns = 'webtoon_id',
                              values = 'rating')

matrix_dummy = rating_matrix.copy().fillna(0) # 원본 손상 방지
user_matrix_dummy = user_matrix.copy().fillna(0)

user_rating_similarity = cosine_similarity(matrix_dummy, matrix_dummy)

user_info_similarity = cosine_similarity(user_matrix_dummy, user_matrix_dummy)

user_rating_similarity = pd.DataFrame(user_rating_similarity,
                               index = rating_matrix.index,
                               columns = rating_matrix.index)
user_info_similarity = pd.DataFrame(user_info_similarity,
                               index = user_matrix.index,
                               columns = user_matrix.index)


user_similarity = user_rating_similarity.add(user_info_similarity)
user_similarity = user_similarity.dropna(how='all')
user_similarity = user_similarity.dropna(how='all', axis=1)
# user_similarity = user_similarity_df.to_numpy()
print(rating_matrix.loc[16])
# pd.DataFrame(user_similarity)


webtoon_id
21815     NaN
64997     4.0
131385    2.0
318995    4.0
400739    3.0
         ... 
801035    NaN
801324    NaN
801446    NaN
801505    NaN
801555    NaN
Name: 16, Length: 245, dtype: float64


In [178]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [179]:
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['webtoon_id']) # 테스트 데이터 간의 user_id와 webtoon_id 페어를 맞춰 튜플화
    y_pred = np.array([model(user,webtoon) for (user,webtoon) in id_pairs]) # 예측값
    print(y_pred)
    y_true = np.array(x_test['rating']) # 테스트데이터의 실제 값
    print(y_true)
    return RMSE(y_true, y_pred)

In [180]:
merged_ratings = pd.merge(x_train, users)
users.set_index('user_id')

MBTI_mean = merged_ratings[['webtoon_id', 'MBTI', 'rating']].groupby(['webtoon_id', 'MBTI'])['rating'].mean()
print(MBTI_mean)

def cf_mbti(user_id, webtoon_id):
    if webtoon_id in rating_matrix.columns:
        MBTI = users.loc[user_id]['MBTI']
        if MBTI in MBTI_mean[webtoon_id].index:
            MBTI_rating = MBTI_mean[webtoon_id][MBTI]
        else:
            MBTI_rating = 3.0
    else:
        MBTI_rating = 3.0
    return MBTI_rating


webtoon_id  MBTI
21815       ESTP    5.0
            INTP    3.0
            ISTJ    3.0
64997       ESTP    4.0
            INTP    3.0
                   ... 
801324      ISFP    4.0
801446      INTP    4.0
801505      ESFP    4.0
            INTP    3.0
801555      ISTJ    5.0
Name: rating, Length: 378, dtype: float64


In [181]:
def CF_knn(user_id, webtoon_id, neighbor_size = 0):
    
    if webtoon_id in rating_matrix.columns:
        sim_scores = user_similarity[user_id].copy()
        webtoon_ratings = rating_matrix[webtoon_id].copy()
        none_rating_idx = webtoon_ratings[webtoon_ratings.isnull()].index
        webtoon_ratings = webtoon_ratings.dropna()
        sim_scores = sim_scores.drop(none_rating_idx)

        if neighbor_size == 0:
            mean_rating = np.dot(sim_scores, webtoon_ratings) / sim_scores.sum()

        else:
            if len(sim_scores) > 1:
                neighbor_size = min(neighbor_size, len(sim_scores)) #sim_score보다 neighbor_size가 클 수 있기 때문에 크기를 맞춰줌.
                sim_scores = np.array(sim_scores)
                webtoon_ratings = np.array(webtoon_ratings)
                user_idx = np.argsort(sim_scores) #sim_scores를 오름차순으로 index값을 추출
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                webtoon_ratings = webtoon_ratings[user_idx][-neighbor_size:] #neighbor_size 만큼 추출
                score_sum = sim_scores.sum()
                if score_sum == 0.0:
                    score_sum = 0.1
                mean_rating = np.dot(sim_scores, webtoon_ratings) / sim_scores.sum() # 행렬 곱

            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
        
    return mean_rating

In [183]:
def recom_webtoon(user_id, n_items, neighbor_size = 30):
    user_webtoon = rating_matrix.loc[user_id].copy()

    for webtoon in rating_matrix.columns:
        if pd.notnull(user_webtoon.loc[webtoon]):
            user_webtoon.loc[webtoon] = 0 # 해당 웹툰을 이미 평가한 경우는 제외.
    
        else:
            user_webtoon.loc[webtoon] = CF_knn(user_id, webtoon, neighbor_size)

    webtoon_sort = user_webtoon.sort_values(ascending=False)[:n_items]
    recom_webtoon = webtoons.loc[webtoon_sort.index]
    recommendations = recom_webtoon
    
    for i in range(n_items):
        if ((recommendation['user']==user_id).any()) == False :
            SQL = "INSERT INTO recommendation SET user=%s, webtoonid=%s, rank=%s"
            SQL_data = (user_id, recommendations.index[i], i+1)
            cur.execute(SQL, SQL_data)
            conn.commit()
        else:
            SQL = "UPDATE recommendation SET user=%s, webtoonid=%s, rank=%s WHERE user=%s and rank=%s"
            SQL_data = (user_id, recommendations.index[i], i+1, user_id, i+1)
            cur.execute(SQL, SQL_data)
            conn.commit()

    return recommendations

recom_webtoon(user_id = 5, n_items = 20, neighbor_size = 30)

,title_name,author,day,genre,story,link,img_src
webtoon_id,,,,,,,
776601,광마회귀,JP / 이히 / 유진성,금,"스토리, 무협/사극",무공에 미친 광마 이자하.\n그는 마교 교주의 천옥을 훔쳐 쫓기던 중 벼랑에서 떨어...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...
739115,앵무살수,김성진,월,"스토리, 판타지",평범한 뱃사공으로 살고 있는 노소하. \n하지만 그의 정체는 전설적인 구파검법의 후...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...
747271,나노마신,현절무 / 금강불괴 / 한중월야,목,"스토리, 무협/사극","갖은 멸시와 목숨의 위협을 받던 마교의 사생아 천여운, \n미래에서 나타난 후손이 ...",https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...
746858,이번 생도 잘 부탁해,이혜,일,"스토리, 로맨스",전생을 기억하는 삶은 어떨까? 매번 전생을 기억하는 무료한 삶을 살던 18회차 인생...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...
730694,초인의 시대,섭이,토,"스토리, 액션",지구에 근원 모를 괴물들이 출현하고,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...
784248,내가 키운 S급들,seri / 비완 / 근서,화,"스토리, 판타지",F급 헌터.\n그것도 잘나가는 S급 동생 발목이나 잡는 쓸모없고 찌질한 F급 형.\...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...
790713,대학원 탈출일지,요다,"화, 금","에피소드, 일상",연구에 꿈을 가지고 대학원을 선택한 병아리 요다.\n예상과는 다르게 연구에 관심 없...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...
753478,데드퀸,김규삼,금,"스토리, 판타지","절망에 빠져 죽음이 임박했을때, 진짜 여왕이 재림한다!\n하이브 세계관의 장대한 끝...",https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...
793283,악몽의 형상,김용키,"화, 토","스토리, 스릴러","'타인은지옥이다', '관계의종말' 이후 9년..\n종우와 다은은 여전히 끔찍한 지옥...",https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...


In [163]:
(recommendation['user']==5).any()

False

In [27]:
def score(model, neighbor_size = 0):
    id_pairs = zip(x_test['user_id'], x_test['webtoon_id'])
    y_pred = np.array([model(user, webtoon, neighbor_size) for (user, webtoon) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

score(CF_knn, neighbor_size = 30)

1.1316594526142632